In [1]:
import torch.nn as nn
import torch.optim as optim
import torch

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator
from torchtext import data
import spacy
import numpy as np

import random
import math
import time

In [2]:
import torch
torch.rand(1).cuda()

tensor([0.5302], device='cuda:0')

In [3]:
SEED = 1234


random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [4]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

In [5]:
def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [ ]:
SRC = Field(tokenize=tokenize_de, init_token='<sos>', eos_token='<eos>',lower=True)

TRG = Field(tokenize=tokenize_en, init_token='<sos>', eos_token='<eos>',lower=True)

In [6]:
text = data.Field()
response = data.Field()


/home/anuda/anaconda3/envs/pytorch/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [7]:


# create tuples representing the columns
fields = [
  ('text', text),
  ('response', response)
]

In [ ]:
# import pandas as pd
# x = pd.read_csv('twitter_data/cleaned_tweets_valid.csv')
# x = x.loc[~pd.isna(x.response)]
# x.shape

In [ ]:
# x.to_csv('twitter_data/cleaned_tweets_valid.csv',index=False)

In [8]:
train_data, valid_data, test_data = data.TabularDataset.splits(
   path = 'twitter_data',
   train = 'cleaned_tweets_train.csv',
   validation = 'cleaned_tweets_valid.csv',
   test = 'cleaned_tweets_test.csv',
   format = 'csv',
   fields = fields,
   skip_header = True
)

/home/anuda/anaconda3/envs/pytorch/lib/python3.6/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/home/anuda/anaconda3/envs/pytorch/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


In [9]:
print(vars(train_data.examples[0]))

{'text': ['115712', 'i', 'understand', 'i', 'would', 'like', 'to', 'assist', 'you', 'we', 'would', 'need', 'to', 'get', 'you', 'into', 'a', 'private', 'secured', 'link', 'to', 'further', 'assist'], 'response': ['sprintcare', 'and', 'how', 'do', 'you', 'propose', 'we', 'do', 'that']}


In [10]:
train_data.examples[1000].__dict__.keys()

dict_keys(['text', 'response'])

#### Cuild Vocab, tokens must have frequency of atleast 2

In [11]:
text.build_vocab(train_data,min_freq=100)
response.build_vocab(train_data,min_freq=100)

In [12]:
len(text.vocab),len(response.vocab)

(7727, 6733)

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [30]:
BATCH_SIZE = 32

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
            (train_data, valid_data, test_data),
            batch_size=BATCH_SIZE,sort=False,
            device=device)

In [31]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        #TO DO: add 5 more asserts
#         assert encoder.hid_dim == decoder.hid_dim,
    
    def forward(self,src, trg,teacher_force_ratio=0.5):
        context = self.encoder(src)
        hidden = context # only first GRU
        batch_size = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim
        input = trg[0,:]
        trg_len = trg.shape[0]
        #decoder needs to run one by one
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device) # assign to device
        for t in range(1, trg_len):
            output, hidden = self.decoder(input, hidden, context)
            outputs[t] = output
            
            teacher_force = random.random() < teacher_force_ratio
            top1 = output.argmax(1)
            input = trg[t] if teacher_force else top1
        return outputs
        

In [32]:
class Encoder(nn.Module):
    def __init__(self,input_dim,emb_dim, hid_dim, dropout):
        
        super().__init__()
        
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim

        self.embedding = nn.Embedding(self.input_dim, self.emb_dim)
        self.rnn = nn.GRU(self.emb_dim, self.hid_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self,src):
        
        embedded = self.dropout(self.embedding(src))
        output,hidden = self.rnn(embedded)
        
        return hidden
        
        

In [33]:
class Decoder(nn.Module):
    def __init__(self,output_dim,emb_dim, hid_dim, dropout):
        
        super().__init__()
        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim, hid_dim, dropout)
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        self.fc_out = nn.Linear(emb_dim+hid_dim*2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    
    def forward(self,input, hidden, context):
        
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        emb_con = torch.cat((embedded, context),dim=2)
        output, hidden = self.rnn(emb_con, hidden)
        output_concatenated = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), dim=1)
        prediction = self.fc_out(output_concatenated)
        
        return prediction, hidden
        
        
        

In [34]:
INPUT_DIM = len(text.vocab)
OUTPUT_DIM = len(response.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)


In [35]:
INPUT_DIM,OUTPUT_DIM

(7727, 6733)

In [36]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7727, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(6733, 256, padding_idx=512, max_norm=0.5)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=6733, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [37]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'the model has {count_parameters(model)} trainable parameters')

the model has 15478605 trainable parameters


In [38]:
optimizer = optim.Adam(model.parameters())

In [39]:
TRG_PAD_IDX = response.vocab.stoi[response.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

In [40]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.text
        trg = batch.response
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [41]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.text
            trg = batch.response

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [42]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time/60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 48m 33s
	Train Loss: 4.487 | Train PPL:  88.885
	 Val. Loss: 5.973 |  Val. PPL: 392.774
Epoch: 02 | Time: 48m 55s
	Train Loss: 4.202 | Train PPL:  66.789
	 Val. Loss: 6.042 |  Val. PPL: 420.826
Epoch: 03 | Time: 48m 54s
	Train Loss: 4.129 | Train PPL:  62.095
	 Val. Loss: 6.035 |  Val. PPL: 417.812
Epoch: 04 | Time: 49m 14s
	Train Loss: 4.086 | Train PPL:  59.497
	 Val. Loss: 6.145 |  Val. PPL: 466.288


In [ ]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

In [ ]:
for i, batch in enumerate(train_iterator): 
    src = batch.src
    trg = batch.trg

    optimizer.zero_grad()

    output = model(src, trg)
    output_dim = output.shape[-1]
            
    output = output[1:].view(-1, output_dim)
    print(np.argmax(output.cpu().detach().numpy(),axis=0))
    break

In [ ]:
trg.shape

In [ ]:
train_data.examples[0].__dict__['src'],train_data.examples[0].__dict__['trg']

In [ ]:
TRG.vocab.stoi